#**IMPORTS**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -U scikit-learn



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#helper function 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics  import f1_score,accuracy_score


def label_encoder(column, df):
    le = LabelEncoder()
    df[column]=le.fit_transform(df[column])
    return 

In [ ]:
#Importing the datasets
test_data="/content/test.csv"
train_data="/content/train.csv"


def import_data(some_csv:str, sep=","):
  return pd.read_csv(some_csv, header=0, sep=sep)

df_train=import_data(train_data)
df_test = import_data(test_data)


df_train=df_train.drop('Unnamed: 0', axis=1)
df_test=df_test.drop('Unnamed: 0', axis=1)

In [ ]:
df_train.isna().sum()

date                   0
org                 3451
tld                 3453
ccs                    0
bcced                  0
mail_type            172
images                 0
urls                   0
salutations            0
designation            0
chars_in_subject      16
chars_in_body          0
label                  0
dtype: int64

## Duplicates

### Duplicates analysis

In [ ]:
df_train[df_train['chars_in_body'] == 13464]

,date,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,label
18701,"Fri, 1 May 2015 11:48:55 +0530 (IST)",NaN,NaN,0,0,text/html,0,2,1,0,73.0,13464,5
24351,"Fri, 1 May 2015 11:48:55 +0530 (IST)",NaN,NaN,0,0,text/html,0,2,1,0,73.0,13464,1
34895,"Fri, 1 May 2015 11:06:59 +0530 (IST)",NaN,NaN,0,0,text/html,0,2,1,0,74.0,13464,5
46704,"Fri, 1 May 2015 10:44:34 +0530 (IST)",NaN,NaN,0,0,text/html,0,2,1,0,73.0,13464,1
58428,"Fri, 1 May 2015 11:06:59 +0530 (IST)",NaN,NaN,0,0,text/html,0,2,1,0,74.0,13464,0
79354,"Fri, 1 May 2015 10:44:34 +0530 (IST)",NaN,NaN,0,0,text/html,0,2,1,0,73.0,13464,5
80172,"Fri, 1 May 2015 11:48:55 +0530 (IST)",NaN,NaN,0,0,text/html,0,2,1,0,73.0,13464,0


There are around 11k duplicates

In [ ]:
df_train[df_train.duplicated()]

,date,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,label
1848,"Fri, 07 Apr 2017 15:24:00 +0530",nrsc,gov.in,0,0,multipart/alternative,0,4,1,0,32.0,1210,0
1883,"Sat, 24 Dec 2016 13:13:33 +0530",iiitd,ac.in,0,0,multipart/mixed,0,1,1,1,6.0,14339248,1
1895,"Fri, 07 Apr 2017 10:32:01 +0530",nrsc,gov.in,0,0,multipart/alternative,0,4,1,0,32.0,1210,1
2046,"Thu, 24 Aug 2017 09:24:00 +0000 (UTC)",researchgatemail,net,0,0,multipart/alternative,2,18,0,1,150.0,21900,7
2142,"Tue, 25 Jun 2019 21:13:12 +0000 (UTC)",github,com,0,0,text/plain,0,3,0,0,52.0,446,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80151,"Sun, 17 Jan 2016 18:28:14 +0000",usebackpack,com,0,0,multipart/alternative,4,19,0,0,37.0,26969,1
80155,"Fri, 18 Nov 2016 14:33:28 +0530",iiitd,ac.in,0,0,multipart/alternative,0,16,1,1,48.0,17329,1
80159,"Wed, 6 Apr 2016 22:57:30 +0530",iiitd,ac.in,5,0,multipart/alternative,0,3,1,1,90.0,8851,3
80160,"Tue, 26 Dec 2017 08:48:49 +0000",twitter,com,0,0,multipart/alternative,43,240,0,1,31.0,95791,7


And 31k that have the same email (without taking the label into account)

In [ ]:
df_dup = df_train.drop('label', axis=1)

df_dup[df_dup.duplicated()]

,date,org,tld,ccs,bcced,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body
232,4 Dec 2013 13:50:13 -0000,phpclasses,org,0,0,multipart/alternative,21,55,0,0,50.0,26592
434,"Thu, 4 Feb 2016 16:00:36 +0530",iiitd,ac.in,2,0,multipart/related,0,0,1,0,96.0,1521139
577,"Sat, 5 Mar 2016 12:23:17 +0530 (IST)",sampark,gov.in,0,0,text/html,52,101,0,0,75.0,28650
985,"Tue, 18 Aug 2015 12:14:47 +0530",iiitd,ac.in,7,0,multipart/alternative,2,13,1,1,34.0,9562
987,"Fri, 07 Apr 2017 15:30:01 +0530",nrsc,gov.in,0,0,multipart/alternative,0,4,1,0,32.0,1210
...,...,...,...,...,...,...,...,...,...,...,...,...
80168,"Fri, 15 Feb 2019 18:07:55 +0100",asvspoof,org,0,0,multipart/alternative,0,7,1,0,29.0,16533
80170,"Tue, 07 Apr 2015 14:12:29 +0000 (UTC)",quora,com,0,0,multipart/alternative,0,88,1,1,98.0,80913
80172,"Fri, 1 May 2015 11:48:55 +0530 (IST)",NaN,NaN,0,0,text/html,0,2,1,0,73.0,13464
80174,"Fri, 14 Dec 2018 09:01:13 +0000 (UTC)",medium,com,0,0,multipart/alternative,32,239,1,1,169.0,105276


#### Dealing with duplicates emails with different labels

In [ ]:
# FOR TRAIN

# there are duplicates ie mail that have more than 1 label.
# to take this info into account, we will add 8 columns, with default value 0, and 1 if we find a duplicate with different label 

dup_dic = dict()

for email in df_train.itertuples(index=False, name = None):
  email_no_lab = email[0:-1]
  email_no_lab = tuple(map(str, email_no_lab))
  # first, we are going to fill a dictionary where key is the label free tuple and value is a list of the label it has received
  if email_no_lab not in dup_dic.keys():
    if email[-1] is None:
      dup_dic[email_no_lab] = []
    else: 
      dup_dic[email_no_lab] = [email[-1]] # we add the label as first value
  elif email_no_lab in dup_dic.keys():
    # part for the test set: in that case, nothing to add because there is no label to add
    if email[-1] is None: 
      pass
    else:
      dup_dic[email_no_lab].append(email[-1]) # we add the label to the list of labels
      dup_dic[email_no_lab] = list(set(dup_dic[email_no_lab]))

# we add 1 column for each of the label
for col in range(8):
  col_name = 'label_' + str(col)
  df_train[col_name] = 0
  for email in df_train.itertuples(name = None):
    index = email[0]
    email_no_lab = email[1:13]
    email_no_lab = tuple(map(str, email_no_lab))
    label = email[-1]
    label_list = dup_dic[email_no_lab]
    if col in label_list:
      df_train.loc[index, col_name] = 1


In [ ]:
#df_train = duplicate_email(df_train)

# we set 0 to label column corresponding to label  
for idx, row in df_train.iterrows():
  label = row['label']
  column = 'label_' + str(label)
  df_train.loc[idx, column] = 0

In [ ]:
# FOR TEST

df_test['label'] = 9 # we initialize label column in test with value not in 0-8

In [ ]:
# there are duplicates ie mail that have more than 1 label.
# to take this info into account, we will add 8 columns, with default value 0, and 1 if we find a duplicate with different label 

for email in df_test.itertuples(index=False, name = None):
  email_no_lab = email[0:-1]
  email_no_lab = tuple(map(str, email_no_lab))
  # first, we are going to fill a dictionary where key is the label free tuple and value is a list of the label it has received
  if email_no_lab not in dup_dic.keys():
    if email[-1] is None:
      dup_dic[email_no_lab] = []
    else: 
      dup_dic[email_no_lab] = [email[-1]] # we add the label as first value
  elif email_no_lab in dup_dic.keys():
    # part for the test set: in that case, nothing to add because there is no label to add
    if email[-1] is None: 
      pass
    else:
      dup_dic[email_no_lab].append(email[-1]) # we add the label to the list of labels
      dup_dic[email_no_lab] = list(set(dup_dic[email_no_lab]))

# we add 1 column for each of the label
for col in range(8):
  col_name = 'label_' + str(col)
  df_test[col_name] = 0
  for email in df_test.itertuples(name = None):
    index = email[0]
    email_no_lab = email[1:13]
    email_no_lab = tuple(map(str, email_no_lab))
    label = email[-1]
    label_list = dup_dic[email_no_lab]
    if col in label_list:
      df_test.loc[index, col_name] = 1


In [ ]:
df_test = df_test.drop('label', axis=1)

In [ ]:
print(df_train.shape, df_test.shape)

(80176, 21) (34365, 20)


### No_url and no_ccs columns

In [ ]:
#Creating a one hot econding column for when there is no url or no css
def no_col(rows, col_name):
    if rows[col_name] != 0:
        return 0
    else:
        return 1

## For train set
df_train["no_url"]= df_train.apply(lambda row: no_col(row, 'urls'), axis=1)
df_train["no_ccs"]= df_train.apply(lambda row: no_col(row, 'ccs'), axis=1)


## For test set
df_test["no_url"]= df_test.apply(lambda row: no_col(row, 'urls'), axis=1)
df_test["no_ccs"]= df_test.apply(lambda row: no_col(row, 'ccs'), axis=1)

### Numerical values: chars_in_subject, chars_in_body, urls, images

In [ ]:
# replacing nas in chars in body by 0
df_train['chars_in_subject'] = df_train['chars_in_subject'].fillna(0)
df_test['chars_in_subject'] = df_test['chars_in_subject'].fillna(0)

In [ ]:
#Discretize numerical variables - quantile bins
#Number of characters in body
df_train['quantile_chars_body'] = pd.qcut(df_train['chars_in_body'],
                              q=[0, .2, .4, .6, .8, 1],
                              labels=[1,2,3,4,5])

df_test['quantile_chars_body'] = pd.qcut(df_test['chars_in_body'],
                              q=[0, .2, .4, .6, .8, 1],
                              labels=[1,2,3,4,5])

#Images
df_train['quantile_images'] = pd.qcut(df_train['images'], q=5, labels=[1,2,3,4], duplicates= 'drop')
df_test['quantile_images'] = pd.qcut(df_test['images'], q=5, labels=[1,2,3,4], duplicates= 'drop')

#Urls
df_train['quantile_urls'] = pd.qcut(df_train['urls'], q=5, labels=[1,2,3,4,5], duplicates= 'drop')
df_test['quantile_urls'] = pd.qcut(df_test['urls'], q=5, labels=[1,2,3,4,5], duplicates= 'drop')

#Number of characters in 
df_train['quantile_chars_subject'] = pd.qcut(df_train['chars_in_subject'], q=5, labels=[1,2,3,4,5], duplicates= 'drop')
df_test['quantile_chars_subject'] = pd.qcut(df_test['chars_in_subject'], q=5, labels=[1,2,3,4,5], duplicates= 'drop')

In [ ]:
#Save in a list the indexes of the row instances where the variables images, urls, chars_in_body and chars_in_subject are equal to 0
no_image = df_train.index[df_train['images'] == 0].tolist()
no_urls = df_train.index[df_train['urls'] == 0].tolist()
no_chars_body = df_train.index[df_train['chars_in_body'] == 0].tolist()
no_chars_sub = df_train.index[df_train['chars_in_subject'] == 0].tolist()

no_image_t = df_test.index[df_test['images'] == 0].tolist()
no_urls_t = df_test.index[df_test['urls'] == 0].tolist()
no_chars_body_t = df_test.index[df_test['chars_in_body'] == 0].tolist()
no_chars_sub_t = df_test.index[df_test['chars_in_subject'] == 0].tolist()


In [ ]:
#Changing the data type of the quantile columns to int type, so that they can be edited
df_train["quantile_images"] = df_train['quantile_images'].astype('int')
df_train["quantile_urls"] = df_train['quantile_urls'].astype('int')
df_train["quantile_chars_body"] = df_train['quantile_chars_body'].astype('int')
df_train["quantile_chars_subject"] = df_train['quantile_chars_subject'].astype('int')

df_test["quantile_images"] = df_test['quantile_images'].astype('int')
df_test["quantile_urls"] = df_test['quantile_urls'].astype('int')
df_test["quantile_chars_body"] = df_test['quantile_chars_body'].astype('int')
df_test["quantile_chars_subject"] = df_test['quantile_chars_subject'].astype('int')

In [ ]:
#Function changing to zero the values at the given row indexes
def change_zeros(list, col,dataframe):
  for ind in list:
    dataframe.at[ind,col]= 0
  return None

#changing the category of the discretized variables to 0, when the regular variable is equal to 0. 
change_zeros(no_image,'quantile_images', df_train)
change_zeros(no_urls,'quantile_urls', df_train)
change_zeros(no_chars_sub,'quantile_chars_subject', df_train)

change_zeros(no_image_t,'quantile_images', df_test)
change_zeros(no_urls_t,'quantile_urls', df_test)
change_zeros(no_chars_sub_t,'quantile_chars_subject', df_test)

In [ ]:
# check if mail is image heavy compared to nb of chars in body
df_train['image_heavy'] = df_train.apply(lambda x: 1 if x['quantile_chars_body'] > x['quantile_images'] else 0, axis=1)
df_test['image_heavy'] = df_test.apply(lambda x: 1 if x['quantile_chars_body'] > x['quantile_images'] else 0, axis=1)

In [ ]:
df_train['log_image_char'] = df_train.apply(lambda x: np.log2(x['chars_in_body'] + 1) + np.log2(x['images'] + 1), axis=1)
df_test['log_image_char'] = df_test.apply(lambda x: np.log2(x['chars_in_body'] + 1) + np.log2(x['images'] + 1), axis=1)

In [ ]:
#droppi
df_train = df_train.drop(['chars_in_body','images','urls','chars_in_subject'], axis=1)

df_test = df_test.drop(['chars_in_body','images','urls','chars_in_subject'], axis=1)


### Datetime 

In [ ]:
# we delete format (GMT+xx:xx) which make todatetime fail
import re
df_train['date'] = df_train['date'].map(lambda x: re.sub(r' \(GMT.*\)','',x))
df_test['date'] = df_test['date'].map(lambda x: re.sub(r' \(GMT.*\)','',x))

In [ ]:
#this means that pandas sees this column as a string, not a datetime, we fix that
df_train['date']=pd.to_datetime(df_train['date'], errors='coerce', infer_datetime_format=True, utc=True)

df_test['date']=pd.to_datetime(df_test['date'], errors='coerce', infer_datetime_format=True, utc=True)

# feature engineering
#df_train.date.head(100)

In [ ]:
#we isolate the day because scammers usually go for 
df_train['day']=df_train['date'].dt.day
df_train['hour']=df_train['date'].dt.hour
df_train['year']=df_train['date'].dt.year
df_train['month']=df_train['date'].dt.month
df_train['weekday']=df_train['date'].dt.dayofweek

df_test['day']=df_test['date'].dt.day
df_test['hour']=df_test['date'].dt.hour
df_test['year']=df_test['date'].dt.year
df_test['month']=df_test['date'].dt.month
df_test['weekday']=df_test['date'].dt.dayofweek

In [ ]:
df_train = df_train.drop(['date'], axis=1)

df_test = df_test.drop(['date'], axis=1)

In [ ]:
df_test

,org,tld,ccs,bcced,mail_type,salutations,designation,label_0,label_1,label_2,...,quantile_images,quantile_urls,quantile_chars_subject,image_heavy,log_image_char,day,hour,year,month,weekday
0,iiitd,ac.in,0,0,multipart/alternative,1,0,0,0,0,...,0,3,1,1,12.137632,1,10,2017,11,2
1,notifications,freelancer.com,0,0,multipart/alternative,1,0,0,0,0,...,3,4,3,1,18.007991,6,1,2015,3,4
2,iiitd,ac.in,1,0,multipart/alternative,1,0,0,0,0,...,0,0,1,1,10.738092,21,9,2016,4,3
3,udacity,com,0,0,multipart/alternative,1,0,0,0,1,...,4,4,5,1,20.034049,6,12,2018,2,1
4,google,com,0,0,text/html,1,0,1,1,0,...,4,5,3,0,18.714960,21,12,2019,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34360,quora,com,0,0,multipart/alternative,1,1,0,1,0,...,3,5,5,1,19.612593,20,2,2016,9,1
34361,shop,sdconnect.in,0,0,multipart/alternative,1,0,0,0,1,...,4,5,4,0,19.150184,19,6,2016,8,4
34362,iiitd,ac.in,0,0,multipart/alternative,1,0,0,1,0,...,1,2,2,1,13.154502,4,11,2016,5,2
34363,tatadocomo,com,0,0,multipart/alternative,1,0,1,0,0,...,0,2,3,1,11.339850,29,9,2014,12,0


### Mail type encoding

In [ ]:
#THIS IS YU ---> ADD THE TEST PART

from sklearn.preprocessing import OneHotEncoder
import numpy as np


#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')
df_mail_type = df_train['mail_type'].str.lower().str.strip().str.split('/', expand=True)
df_mail_type_test = df_test['mail_type'].str.lower().str.strip().str.split('/', expand=True)

# to make sure the value before "/" and after don't repeat
mail_type_list0 = list(set(df_mail_type[0].tolist()))
mail_type_list1 = list(set(df_mail_type[1].tolist()))
mail_type_list0 = [str(x) for x in mail_type_list0]
mail_type_list1 = [str(x) for x in mail_type_list1]

mail_type_list0_t = list(set(df_mail_type_test[0].tolist()))
mail_type_list1_t = list(set(df_mail_type_test[1].tolist()))
mail_type_list0_t = [str(x) for x in mail_type_list0_t]
mail_type_list1_t = [str(x) for x in mail_type_list1_t]




i = mail_type_list0.index('nan')
i_t = mail_type_list0_t.index('nan')
  
# replace Rahul with Shikhar
mail_type_list0 = mail_type_list0[:i]+['not_text_multipart']+mail_type_list0[i+1:]
mail_type_list0_t = mail_type_list0_t[:i]+['not_text_multipart']+mail_type_list0_t[i+1:]

i = mail_type_list1.index('nan')
i = mail_type_list1_t.index('nan')
  
# replace Rahul with Shikhar
mail_type_list1 = mail_type_list1[:i]+['others_mail_type']+mail_type_list1[i+1:]
mail_type_list1_t = mail_type_list1_t[:i]+['others_mail_type']+mail_type_list1_t[i+1:]

print(mail_type_list0)
print(mail_type_list1)


# encode the first column
encoder_df1 = pd.DataFrame(encoder.fit_transform(df_mail_type[[0]]).toarray())
encoder_df1.columns = mail_type_list0

# encode the second column
encoder_df2 = pd.DataFrame(encoder.fit_transform(df_mail_type[[1]]).toarray())
encoder_df2.columns = mail_type_list1

#test
encoder_df1_t = pd.DataFrame(encoder.fit_transform(df_mail_type_test[[0]]).toarray())
encoder_df1_t.columns = mail_type_list0_t

# encode the second column
encoder_df2_t = pd.DataFrame(encoder.fit_transform(df_mail_type_test[[1]]).toarray())
encoder_df2_t.columns = mail_type_list1_t

# combine them back to df_train
df_train = pd.concat([df_train, encoder_df1, encoder_df2], axis=1)
df_train.drop('mail_type', axis=1, inplace=True)
df_train.drop("not_text_multipart", axis=1, inplace=True)
df_train.drop("others_mail_type", axis=1, inplace=True)

df_test = pd.concat([df_test, encoder_df1_t, encoder_df2_t], axis=1)
df_test.drop('mail_type', axis=1, inplace=True)
df_test.drop("not_text_multipart", axis=1, inplace=True)
df_test.drop("others_mail_type", axis=1, inplace=True)



['multipart', 'not_text_multipart', 'text']
['others_mail_type', 'signed', 'idm', 'related', 'plain', 'alternative', 'calendar', 'report', 'html', 'mixed']


In [ ]:
print(df_train.shape)
print(df_test.shape)


(80176, 39)
(34365, 37)


In [ ]:
df_test.head()

,org,tld,ccs,bcced,salutations,designation,label_0,label_1,label_2,label_3,...,multipart,text,signed,idm,related,plain,alternative,report,html,mixed
0,iiitd,ac.in,0,0,1,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,notifications,freelancer.com,0,0,1,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,iiitd,ac.in,1,0,1,0,0,0,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,udacity,com,0,0,1,0,0,0,1,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,google,com,0,0,1,0,1,1,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_train.head()

,org,tld,ccs,bcced,salutations,designation,label,label_0,label_1,label_2,...,text,signed,idm,related,plain,alternative,calendar,report,html,mixed
0,reply,ebay.in,0,0,0,0,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,edm,efinmail.com,0,0,0,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,usebackpack,com,0,0,0,0,1,1,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,granular,ai,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,github,com,1,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for i in list(df_train):
  if i not in list(df_test):
    print(i, type(i))
  

label <class 'str'>
calendar <class 'str'>


In [ ]:
#droping calendar because it is not in the df_test
df_train = df_train.drop('calendar', axis=1)

In [ ]:
for i in list(df_test):
  if i not in list(df_train):
    print(i, type(i))

In [ ]:
print(df_train.shape)
print(df_test.shape)

(80176, 38)
(34365, 37)


In [ ]:
df_train.isna().sum()

org                       3451
tld                       3453
ccs                          0
bcced                        0
salutations                  0
designation                  0
label                        0
label_0                      0
label_1                      0
label_2                      0
label_3                      0
label_4                      0
label_5                      0
label_6                      0
label_7                      0
no_url                       0
no_ccs                       0
quantile_chars_body          0
quantile_images              0
quantile_urls                0
quantile_chars_subject       0
image_heavy                  0
log_image_char               0
day                          0
hour                         0
year                         0
month                        0
weekday                      0
multipart                    0
text                         0
signed                       0
idm                          0
related 

In [ ]:
df_test.isna().sum()

org                       1536
tld                       1536
ccs                          0
bcced                        0
salutations                  0
designation                  0
label_0                      0
label_1                      0
label_2                      0
label_3                      0
label_4                      0
label_5                      0
label_6                      0
label_7                      0
no_url                       0
no_ccs                       0
quantile_chars_body          0
quantile_images              0
quantile_urls                0
quantile_chars_subject       0
image_heavy                  0
log_image_char               0
day                          0
hour                         0
year                         0
month                        0
weekday                      0
multipart                    0
text                         0
signed                       0
idm                          0
related                      0
plain   

### tld and org encoding and feature addition

In [ ]:
df_train

,org,tld,ccs,bcced,salutations,designation,label,label_0,label_1,label_2,...,multipart,text,signed,idm,related,plain,alternative,report,html,mixed
0,reply,ebay.in,0,0,0,0,2,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,edm,efinmail.com,0,0,0,0,1,1,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,usebackpack,com,0,0,0,0,1,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,granular,ai,0,0,0,0,1,0,0,0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,github,com,1,0,0,0,1,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80171,github,com,1,0,0,0,3,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80172,NaN,NaN,0,0,1,0,0,0,1,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
80173,google,com,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80174,medium,com,0,0,1,1,0,0,1,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Target encoding

In [ ]:
# dealing with nas in tld and org

df_train['tld'] = df_train['tld'].fillna('not_defined')
df_train['org'] = df_train['org'].fillna('not_defined')


df_test['tld'] = df_test['tld'].fillna('not_defined')
df_test['org'] = df_test['org'].fillna('not_defined')

In [ ]:
df_train

,org,tld,ccs,bcced,salutations,designation,label,label_0,label_1,label_2,...,multipart,text,signed,idm,related,plain,alternative,report,html,mixed
0,reply,ebay.in,0,0,0,0,2,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,edm,efinmail.com,0,0,0,0,1,1,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,usebackpack,com,0,0,0,0,1,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,granular,ai,0,0,0,0,1,0,0,0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,github,com,1,0,0,0,1,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80171,github,com,1,0,0,0,3,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80172,not_defined,not_defined,0,0,1,0,0,0,1,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
80173,google,com,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80174,medium,com,0,0,1,1,0,0,1,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# function to encode tld and org
def target_encoder(df_train, column_):
    temp_df = df_train.copy()
    temp_df['type'] = 1

    temp_df["label_name"] = temp_df['label'].map({0:"Updates_"+column_,1:"Personal_"+column_, 2:"Promotions_"+column_, 3:"Forums_"+column_, 4:"Purchases_"+column_,5:"Travel_"+column_,6:"Spam_"+column_,7:"Social_"+column_})
    
    temp_df_pivot = temp_df.pivot(columns = 'label_name', values = 'type').fillna(0)
    
    temp_df = temp_df.join(temp_df_pivot)
    
    temp_df = temp_df[[column_, "Updates_"+column_,"Personal_"+column_, "Promotions_"+column_, "Forums_"+column_, "Purchases_"+column_,"Travel_"+column_,"Spam_"+column_,"Social_"+column_]].groupby(column_).mean()

    return temp_df

In [ ]:
df_train.head()

,org,tld,ccs,bcced,salutations,designation,label,label_0,label_1,label_2,...,multipart,text,signed,idm,related,plain,alternative,report,html,mixed
0,reply,ebay.in,0,0,0,0,2,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,edm,efinmail.com,0,0,0,0,1,1,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,usebackpack,com,0,0,0,0,1,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,granular,ai,0,0,0,0,1,0,0,0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,github,com,1,0,0,0,1,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
temp_df = target_encoder(df_train, "org")

df_train = pd.merge(df_train, temp_df,on = "org", how= "left")
df_test = pd.merge(df_test, temp_df,on = "org", how= "left")


temp_df = target_encoder(df_train, "tld")

df_train = pd.merge(df_train, temp_df,on = "tld", how= "left")
df_test = pd.merge(df_test, temp_df,on = "tld", how= "left")

In [ ]:
na_col_train = df_train.columns[df_train.isna().any()]
na_col_test = df_test.columns[df_test.isna().any()]

print('train: ', na_col_train)
print('test: ', na_col_test)

train:  Index([], dtype='object')
test:  Index(['Updates_org', 'Personal_org', 'Promotions_org', 'Forums_org',
       'Purchases_org', 'Travel_org', 'Spam_org', 'Social_org', 'Updates_tld',
       'Personal_tld', 'Promotions_tld', 'Forums_tld', 'Purchases_tld',
       'Travel_tld', 'Spam_tld', 'Social_tld'],
      dtype='object')


In [ ]:
# dealing with nas 

for i in na_col_test.tolist():
  df_test[i] = df_test[i].fillna(0)

In [ ]:
#we add a dot to the top level domain that are shorter than 3 strings, this helps us isolate the country code top-level domains 
# Meaning that we can now match some records with a location 
#df_train['tld']=np.where(df_train['tld'].map(str).apply(len)<3,"."+df_train['tld'].map(str),df_train['tld'].map(str))
#df_test['tld']=np.where(df_test['tld'].map(str).apply(len)<3,"."+df_test['tld'].map(str),df_test['tld'].map(str))
#df_train.groupby(['tld']).size()

In [ ]:
print(df_train.shape, df_test.shape)

(80176, 54) (34365, 53)


In [ ]:
for i in list(df_train):
  if i not in list(df_test):
    print(i, type(i))

label <class 'str'>


In [ ]:
df_train.isna().sum()

org                       0
tld                       0
ccs                       0
bcced                     0
salutations               0
designation               0
label                     0
label_0                   0
label_1                   0
label_2                   0
label_3                   0
label_4                   0
label_5                   0
label_6                   0
label_7                   0
no_url                    0
no_ccs                    0
quantile_chars_body       0
quantile_images           0
quantile_urls             0
quantile_chars_subject    0
image_heavy               0
log_image_char            0
day                       0
hour                      0
year                      0
month                     0
weekday                   0
multipart                 0
text                      0
signed                    0
idm                       0
related                   0
plain                     0
alternative               0
report              

#### New columns

In [ ]:
#THIS IS LUCRE 

def academic_institution(rows):
    if 'ac.' in str(rows['tld']):
        return 1
    if 'edu' in str(rows['tld']):
        return 1
    if 'centralesupelec' in str(rows['org']):
        return 1
    if 'iiitd' in str(rows['org']):
        return 1
    else:
        return 0
    
df_train.apply(lambda row: academic_institution(row), axis=1)

df_train["academic"]= df_train.apply(lambda row: academic_institution(row), axis=1)

df_test.apply(lambda row: academic_institution(row), axis=1)

df_test["academic"]= df_test.apply(lambda row: academic_institution(row), axis=1)

def gov_institution(rows):
    if 'gov' in str(rows['tld']):
        return 1
    else:
        return 0
    
df_train.apply(lambda row: gov_institution(row), axis=1)

df_train["government"]= df_train.apply(lambda row: gov_institution(row), axis=1)

df_test.apply(lambda row: gov_institution(row), axis=1)

df_test["government"]= df_test.apply(lambda row: gov_institution(row), axis=1)

def e_learning(rows):
    courses = ['coursera','phpclasses','udacity']
    if any(x in str(rows['org']) for x in courses):
        return 1
    else:
        return 0
    
df_train.apply(lambda row: e_learning(row), axis=1)

df_train["elearning"]= df_train.apply(lambda row: e_learning(row), axis=1)

df_test.apply(lambda row: e_learning(row), axis=1)

df_test["elearning"]= df_test.apply(lambda row: e_learning(row), axis=1)

def travel(rows):
    traveling = ['airfrance', 'booking','goindigo','makemytrip', 'yatra']
    if any(x in str(rows['org']) for x in traveling):
        return 1
    else:
        return 0

df_train.apply(lambda row: travel(row), axis=1)

df_train["traveling"]= df_train.apply(lambda row: travel(row), axis=1)

df_test.apply(lambda row: travel(row), axis=1)

df_test["traveling"]= df_test.apply(lambda row: travel(row), axis=1)

In [ ]:
print(df_test.shape)
print(df_train.shape)

(34365, 57)
(80176, 58)


In [ ]:
for i in list(df_train):
  if i not in list(df_test):
    print(i)

label


In [ ]:
df_train = df_train.drop(['tld', 'org'], axis = 1)
df_test = df_test.drop(['tld', 'org'], axis = 1)

In [ ]:
print(df_test.shape)
print(df_train.shape)

(34365, 55)
(80176, 56)


In [ ]:
for i in list(df_test):
  if i not in list(df_train):
    df_train[i] = 0.0


In [ ]:
for e in list(df_train):
  if e not in list(df_test):
    df_test[e] =0.0

In [ ]:
df_test = df_test.drop(['label'], axis = 1)

In [ ]:
if 'label' in list(df_test):
  print('error')
else:
  print('no error')

no error


In [ ]:
print(df_test.shape)
print(df_train.shape)

(34365, 55)
(80176, 56)


In [ ]:
df_test

,ccs,bcced,salutations,designation,label_0,label_1,label_2,label_3,label_4,label_5,...,Promotions_tld,Forums_tld,Purchases_tld,Travel_tld,Spam_tld,Social_tld,academic,government,elearning,traveling
0,0,0,1,0,0,0,0,0,0,0,...,0.000549,0.433939,0.000399,0.000449,0.0000,0.00000,1,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.00000,0,0,0,0
2,1,0,1,0,0,0,0,1,0,0,...,0.000549,0.433939,0.000399,0.000449,0.0000,0.00000,1,0,0,0
3,0,0,1,0,0,0,1,0,0,0,...,0.104625,0.037812,0.001704,0.001473,0.0013,0.12834,0,0,1,0
4,0,0,1,0,1,1,0,0,0,0,...,0.104625,0.037812,0.001704,0.001473,0.0013,0.12834,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34360,0,0,1,1,0,1,0,0,0,0,...,0.104625,0.037812,0.001704,0.001473,0.0013,0.12834,0,0,0,0
34361,0,0,1,0,0,0,1,0,0,0,...,0.500000,0.000000,0.000000,0.000000,0.0000,0.00000,0,0,0,0
34362,0,0,1,0,0,1,0,0,0,0,...,0.000549,0.433939,0.000399,0.000449,0.0000,0.00000,1,0,0,0
34363,0,0,1,0,1,0,0,0,0,0,...,0.104625,0.037812,0.001704,0.001473,0.0013,0.12834,0,0,0,0


In [ ]:
df_train.isna().sum()

ccs                       0
bcced                     0
salutations               0
designation               0
label                     0
label_0                   0
label_1                   0
label_2                   0
label_3                   0
label_4                   0
label_5                   0
label_6                   0
label_7                   0
no_url                    0
no_ccs                    0
quantile_chars_body       0
quantile_images           0
quantile_urls             0
quantile_chars_subject    0
image_heavy               0
log_image_char            0
day                       0
hour                      0
year                      0
month                     0
weekday                   0
multipart                 0
text                      0
signed                    0
idm                       0
related                   0
plain                     0
alternative               0
report                    0
html                      0
mixed               

In [ ]:
df_test.isna().sum()

ccs                       0
bcced                     0
salutations               0
designation               0
label_0                   0
label_1                   0
label_2                   0
label_3                   0
label_4                   0
label_5                   0
label_6                   0
label_7                   0
no_url                    0
no_ccs                    0
quantile_chars_body       0
quantile_images           0
quantile_urls             0
quantile_chars_subject    0
image_heavy               0
log_image_char            0
day                       0
hour                      0
year                      0
month                     0
weekday                   0
multipart                 0
text                      0
signed                    0
idm                       0
related                   0
plain                     0
alternative               0
report                    0
html                      0
mixed                     0
Updates_org         

### Label Encoding

In [ ]:
list_of_ord_variables=("day","hour", "year", "month", "weekday",'quantile_chars_body','quantile_images','quantile_urls','quantile_chars_subject')
for var in list_of_ord_variables:
    label_encoder(var, df_train)
    label_encoder(var, df_test)

In [ ]:
label=df_train['label']
df_train.pop('label')
df_train=pd.concat([df_train, label], axis=1)

In [ ]:
if 'label' in list(df_train)[:-1]:
  print('error')
else:
  print('no error')

no error


In [ ]:
df_test = df_test[list(df_train)[:-1]]


In [ ]:
print(df_test.shape)
print(df_train.shape)

(34365, 55)
(80176, 56)


In [ ]:
for ind in range(df_test.shape[1]):
  if list(df_test)[ind] != list(df_train)[ind]:
    print('in the test:', list(df_test)[ind], '\tIn the train:',list(df_train)[ind] )

In [ ]:
df_train.isna().sum().sort_values(ascending =False)

ccs                       0
bcced                     0
plain                     0
alternative               0
report                    0
html                      0
mixed                     0
Updates_org               0
Personal_org              0
Promotions_org            0
Forums_org                0
Purchases_org             0
Travel_org                0
Spam_org                  0
Social_org                0
Updates_tld               0
Personal_tld              0
Promotions_tld            0
Forums_tld                0
Purchases_tld             0
Travel_tld                0
Spam_tld                  0
Social_tld                0
academic                  0
government                0
elearning                 0
traveling                 0
related                   0
idm                       0
signed                    0
no_ccs                    0
salutations               0
designation               0
label_0                   0
label_1                   0
label_2             

# **END OF FEATURE ENGINEERING**

In [ ]:
df_train = df_train.reset_index()

In [ ]:
y= df_train.label
x = df_train.drop('label', axis =1)
#x = x.drop('label_name', axis=1)


In [ ]:
x.head()

,index,ccs,bcced,salutations,designation,label_0,label_1,label_2,label_3,label_4,...,Promotions_tld,Forums_tld,Purchases_tld,Travel_tld,Spam_tld,Social_tld,academic,government,elearning,traveling
0,0,0,0,0,0,0,0,0,0,0,...,0.474490,0.000000,0.000000,0.000000,0.0000,0.00000,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,...,0.052239,0.000000,0.000000,0.000000,0.0000,0.00000,0,0,0,0
2,2,0,0,0,0,1,0,0,0,0,...,0.104625,0.037812,0.001704,0.001473,0.0013,0.12834,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0.068056,0.187500,0.000000,0.000000,0.0000,0.00000,0,0,0,0
4,4,1,0,0,0,0,0,0,1,0,...,0.104625,0.037812,0.001704,0.001473,0.0013,0.12834,0,0,0,0


In [ ]:
df_test

,index,ccs,bcced,salutations,designation,label_0,label_1,label_2,label_3,label_4,...,Promotions_tld,Forums_tld,Purchases_tld,Travel_tld,Spam_tld,Social_tld,academic,government,elearning,traveling
0,0,0,0,1,0,0,0,0,0,0,...,0.000549,0.433939,0.000399,0.000449,0.0000,0.00000,1,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.00000,0,0,0,0
2,2,1,0,1,0,0,0,0,1,0,...,0.000549,0.433939,0.000399,0.000449,0.0000,0.00000,1,0,0,0
3,3,0,0,1,0,0,0,1,0,0,...,0.104625,0.037812,0.001704,0.001473,0.0013,0.12834,0,0,1,0
4,4,0,0,1,0,1,1,0,0,0,...,0.104625,0.037812,0.001704,0.001473,0.0013,0.12834,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34360,34360,0,0,1,1,0,1,0,0,0,...,0.104625,0.037812,0.001704,0.001473,0.0013,0.12834,0,0,0,0
34361,34361,0,0,1,0,0,0,1,0,0,...,0.500000,0.000000,0.000000,0.000000,0.0000,0.00000,0,0,0,0
34362,34362,0,0,1,0,0,1,0,0,0,...,0.000549,0.433939,0.000399,0.000449,0.0000,0.00000,1,0,0,0
34363,34363,0,0,1,0,1,0,0,0,0,...,0.104625,0.037812,0.001704,0.001473,0.0013,0.12834,0,0,0,0


## Feature selection

In [ ]:
df_test = df_test.reset_index()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=2)

In [ ]:
X_train.shape

(60132, 56)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xg = XGBClassifier()

In [ ]:
xg.fit(X_train, y_train)
score = xg.score(X_test, y_test)
print(score)

0.8659948114148872


In [ ]:
score

0.8659948114148872

In [ ]:
importance = xg.feature_importances_
len(importance)


56

In [ ]:
len(df_train.columns.tolist())

57

In [ ]:
from numpy import sort

#determine the most important features in the model
importance = xg.feature_importances_
#create dataframe with the features and the related importance
d = {'feature':list(df_train)[:-1],'importance':importance}
imp = pd.DataFrame(data = d)
imp = imp.sort_values(by='importance',ascending = False)
t_imp =imp[imp['importance']>0.001]

In [ ]:
imp

,feature,importance
6,label_1,0.168927
39,Forums_org,0.133695
36,Updates_org,0.110912
43,Social_org,0.097534
38,Promotions_org,0.085750
12,label_7,0.052368
7,label_2,0.047347
5,label_0,0.044835
8,label_3,0.042868
37,Personal_org,0.032103


In [ ]:
top_feature = t_imp.feature.unique().tolist()

In [ ]:
x_top_feat = df_train[top_feature]
X_train, X_test, y_train, y_test = train_test_split(x_top_feat, y, test_size=0.25, random_state=2)

In [ ]:
xg_top  = XGBClassifier()
xg_top.fit(X_train, y_train)
score = xg_top.score(X_test, y_test)

In [ ]:
score

0.8652963480343245

In [ ]:
# evaluate the model by using repeated classified k fold cross-validation, with three repeats and 10 folds.
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(xg_top, x_top_feat, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.604 (0.005)


## CV to determine best params

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
#apply the xgb classifier to the restricted set with cross validation to determine the most important values
xg  = XGBClassifier()

params = {
 'learning_rate' : [0.05,0.10,0.15,0.20,0.25,0.30],
 'max_depth' : [ 3, 4, 5, 6, 8, 10, 12, 15],
 'min_child_weight' : [ 1, 3, 5, 7 ],
 'gamma': [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 'colsample_bytree' : [ 0.3, 0.4, 0.5 , 0.7 ]
}

rs_model=RandomizedSearchCV(xg,param_distributions=params,n_iter=5,n_jobs=-1,cv=5,verbose=3)
rs_model.fit(X_train, y_train)
rs_model.best_estimator_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


XGBClassifier(colsample_bytree=0.5, gamma=0.2, learning_rate=0.05, max_depth=10,
              min_child_weight=3, objective='multi:softprob')

In [ ]:
xg_cv = rs_model.best_estimator_
xg_cv.fit(X_train, y_train)

XGBClassifier(colsample_bytree=0.5, gamma=0.2, learning_rate=0.05, max_depth=10,
              min_child_weight=3, objective='multi:softprob')

In [ ]:
score2 = xg_cv.score(X_test, y_test)
print(score2)

0.8877469566952704


In [ ]:
# evaluate the model by using repeated classified k fold cross-validation, with three repeats and 10 folds.
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(xg_cv, x, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.589 (0.005)


# Final

In [ ]:
x_t = df_test[top_feature]

In [ ]:
x_t.isna().sum()

label_1                   0
Forums_org                0
Updates_org               0
Social_org                0
Promotions_org            0
label_7                   0
label_2                   0
label_0                   0
label_3                   0
Personal_org              0
alternative               0
year                      0
ccs                       0
quantile_chars_subject    0
Updates_tld               0
Promotions_tld            0
label_4                   0
Social_tld                0
Purchases_org             0
government                0
no_url                    0
log_image_char            0
idm                       0
weekday                   0
Spam_tld                  0
Spam_org                  0
salutations               0
Travel_org                0
multipart                 0
month                     0
quantile_urls             0
designation               0
quantile_images           0
hour                      0
label_5                   0
Purchases_tld       

In [ ]:
for n in list(x_t):
  if n == 'index':
    print('index')

In [ ]:
print(len(df_test.to_numpy()[0]))

56


In [ ]:
#x_t = selection.transform(df_test.to_numpy())

In [ ]:
pred = xg_cv.predict(x_t)

In [ ]:
print(pred)

[1 0 1 ... 3 1 1]


In [ ]:
df_test['label'] = pred


In [ ]:
final = df_test[['label']]

In [ ]:
final.to_csv("final_sub_duplicates.csv", index=True, index_label='Id')

In [ ]:
final

,label
0,1
1,1
2,1
3,1
4,0
...,...
34360,7
34361,2
34362,1
34363,0


#**END OF CODE**

In [ ]:

from sklearn.model_selection import RandomizedSearchCV

xg  = XGBClassifier()

params = {
 'learning_rate' : [0.05,0.10,0.15,0.20,0.25,0.30],
 'max_depth' : [ 3, 4, 5, 6, 8, 10, 12, 15],
 'min_child_weight' : [ 1, 3, 5, 7 ],
 'gamma': [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 'colsample_bytree' : [ 0.3, 0.4, 0.5 , 0.7 ]
}

rs_model=RandomizedSearchCV(xg,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)
rs_model.fit(X_train, y_train)
rs_model.best_estimator_

In [ ]:
rs_model.fit(X_train, y_train)
score = xg.score(X_test, y_test)
print(score)

In [ ]:
label=df_train['label']
df_train.pop('label')
df_train=pd.concat([df_train, label], axis=1)
df_train.shape

In [ ]:
#turn categorical variables into integer labels with sklearn
# some variables are ordinal (i.e. hour) and some are categorical, we don't use the same encoder for both
#list_of_cat_variables=( "tld", "org")
list_of_ord_variables=("day","hour", "year", "month", "weekday",'quantile_chars_body','quantile_images','quantile_urls','quantile_chars_subject')

''''
for var in list_of_cat_variables:
    label_encoder(var, df_train)''''

for var in list_of_ord_variables:
    label_encoder(var, df_train)



In [ ]:
oneh=df_train[['text', 'multipart','related', 'report', 'alternative', 'plain', 'html', 'calendar', 'mixed', 'idm', 'signed','bcced', 'salutations', 'designation','academic', 'government', 'elearning','traveling','no_url','no_ccs']]

categ=df_train[["tld","org","day","hour", "year", "month", "weekday", 'quantile_chars_body','quantile_images','quantile_urls','quantile_chars_subject']]

numeric=df_train[["ccs"]]



label=df_train['label']

df_train=pd.concat([oneh,categ, numeric, label], axis=1)



In [ ]:
df_train

In [ ]:
df_train = df_train.loc[:,~df_train.columns.duplicated()].copy()

df_train.shape

In [ ]:
df_train

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(X_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_test, y_test)))



In [ ]:
from sklearn.model_selection import cross_val_score
s = cross_val_score(XGBClassifier(),x, y)
print(s)

In [ ]:
#to avoid input bias, we reset indexes

df_train=df_train.sample(frac=1, random_state=0)
df_train.reset_index(drop=True, inplace=True)

split_index=int(len(df_train)*0.75)

train_df=df_train[:split_index]
test_df=df_train[split_index:]

#df_train

In [ ]:
X_train=train_df.to_numpy()[:, :-1]
y_train=train_df.to_numpy()[:,-1]

X_test=test_df.to_numpy()[:, :-1]
y_test=test_df.to_numpy()[:, -1]

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)



In [ ]:
feature_types=['Categorical']*31+['Numerical']*1

In [ ]:
!sudo apt-get install build-essential swig
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
import autosklearn.classification

In [ ]:
from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test=train_test_split()

In [ ]:
autosklearn_classifier = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=1800,
    per_run_time_limit=200,
)

autosklearn_classifier.fit(X_train, y_train, feat_type=feature_types)

print(autosklearn_classifier.leaderboard())

In [ ]:
from sklearn.metrics import classification_report 

print(classification_report(y_train, autosklearn_classifier.predict(X_train)))

In [ ]:
print(classification_report(y_test, autosklearn_classifier.predict(X_test)))

In [ ]:
from sklearn.metrics import classification_report 

autosklearn_classifier.show_models()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
predictions = autosklearn_classifier.predict(X_test)
print("Accuracy score:", accuracy_score(y_test, predictions))